In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.inputCollector import inputCollector

# XS Generation for a 3D PWR Assembly
1. Homogenized few group parameters will be generated for a standard 3D PWR assembly
2. The assembly will experience no burnup
3. The assembly will experience no perturbations
4. The assembly will have predefined values for the axial fuel temperatures and coolant densities
taken from a thermo-hydraulic solver
5. The assembly has reflector on the top and bottom axial layers
6. The XS generation will be done using a 2 group structure where the XS will be generated for the fuel and reflector universes

## Assembly Geometry
1. The Assembly will consist of a 19 x 19 sqaure lattice
2. Each channel will have 22 axial layers, with the 1st and 22nd layer being refectors.
3. Each axial layer will have its own material and conditions pre determined by a thermal hyrdaulic solver.


In [2]:
nAxLayers = 22 #number of axial layers

#Pre generated fuel temps and coolant densities
fuelTemps = [
    669.78
    ,725.41
    ,787.58
    ,838.7
    ,878.48
    ,907.2
    ,925.58
    ,935.24
    ,937.28
    ,932.89
    ,922.53
    ,907.06
    ,887.28
    ,863.37
    ,835.66
    ,803.81
    ,769.06
    ,731.00
    ,690.12
    ,657.53
]

coolDens = [
    -0.7542
    ,-0.75224
    ,-0.74954
    ,-0.74623
    ,-0.74244
    ,-0.73828
    ,-0.73387
    ,-0.72931
    ,-0.72471
    ,-0.72013
    ,-0.71565
    ,-0.71136
    ,-0.70729
    ,-0.70353
    ,-0.70013
    ,-0.69715
    ,-0.69464
    ,-0.69265
    ,-0.69123
    ,-0.69027
]

## Generating Materials
1. Since 2 of the 22 axial layers will be reflectors, this means we will have 20 active layers. 
2. The thermohydraulic solver has pretedetermined the temperatures of the 20 active layers for the fuel and coolant densities.
3. For the fuel we will be using UO2 and lightWater for the coolant. 

In [3]:
fuelMats = [None]*len(fuelTemps)
coolMats = [None]*len(fuelTemps)

for i in range(0, len(fuelTemps)):
    fuelMats[i] = MATLIB['UO2'].duplicateMat("f"+ str(i+1))
    fuelMats[i].set('temp', fuelTemps[i])
    coolMats[i] = MATLIB['lightWater'].duplicateMat("m"+ str(i+1))
    coolMats[i].set('dens', coolDens[i])
    
c1 = MATLIB['Zr'].duplicateMat("clad") # Zr cladding
r1 = MATLIB['plenum'].duplicateMat("reflector") # reflector plenum
matList = mats()
matList.addMats(fuelMats)
matList.addMats(coolMats)
matList.addMat(c1) # Zr cladding
matList.addMat(r1) # reflector plenum
print(matList.toString())

mat f1    -10.2141  burn 1 tmp 669.78 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f2    -10.2141  burn 1 tmp 725.41 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f3    -10.2141  burn 1 tmp 787.58 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f4    -10.2141  burn 1 tmp 838.7 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f5    -10.2141  burn 1 tmp 878.48 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f6    -10.2141  burn 1 tmp 907.2 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f7    -10.2141  burn 1 tmp 925.58 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

mat f8    -10.2141  burn 1 tmp 935.24 rgb 255 165   0
92235.09c	0.000815077056
92238.09c	0.0221889771
8016.09c	0.0460081082

ma

## Generating Individual Channel Axial Layers
1. To model an individual channel axial layer we will be using the serpent pin geometry
2. Each indivial pin will have its own set of materials
3. There will be 4 types of pin models, fuel, guide, gap, and reflector pins


In [4]:
pinRadii = np.array([.4095, .4750]) #individual channel layer material region dimensions

fuelPins = []
guidePins = []
gapPins = []

for i in range(0, len(fuelMats)):
    fuelPins.append(pin("F" +str(i+1), 3))
    fuelPins[i].set('materials', np.array([fuelMats[i], c1, coolMats[i]]))
    fuelPins[i].set('radii', pinRadii)
    
    guidePins.append(pin("G" +str(i+1), 3))
    guidePins[i].set('materials', np.array([coolMats[i], c1, coolMats[i]]))
    guidePins[i].set('radii', pinRadii)
    
    gapPins.append(pin("GAP" +str(i+1), 1))
    gapPins[i].set('materials', np.array([coolMats[i]]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([r1]))

RB = pin("RB", 1) #bottom reflector
RB.set('materials', np.array([r1]))

pinList = pins()
pinList.addPins(fuelPins)
pinList.addPins(guidePins)
pinList.addPins(gapPins)
pinList.addPin(RT)
pinList.addPin(RB)

print(pinList.toString())

pin F1
f1	0.4095
clad	0.475
m1

pin F2
f2	0.4095
clad	0.475
m2

pin F3
f3	0.4095
clad	0.475
m3

pin F4
f4	0.4095
clad	0.475
m4

pin F5
f5	0.4095
clad	0.475
m5

pin F6
f6	0.4095
clad	0.475
m6

pin F7
f7	0.4095
clad	0.475
m7

pin F8
f8	0.4095
clad	0.475
m8

pin F9
f9	0.4095
clad	0.475
m9

pin F10
f10	0.4095
clad	0.475
m10

pin F11
f11	0.4095
clad	0.475
m11

pin F12
f12	0.4095
clad	0.475
m12

pin F13
f13	0.4095
clad	0.475
m13

pin F14
f14	0.4095
clad	0.475
m14

pin F15
f15	0.4095
clad	0.475
m15

pin F16
f16	0.4095
clad	0.475
m16

pin F17
f17	0.4095
clad	0.475
m17

pin F18
f18	0.4095
clad	0.475
m18

pin F19
f19	0.4095
clad	0.475
m19

pin F20
f20	0.4095
clad	0.475
m20

pin G1
m1	0.4095
clad	0.475
m1

pin G2
m2	0.4095
clad	0.475
m2

pin G3
m3	0.4095
clad	0.475
m3

pin G4
m4	0.4095
clad	0.475
m4

pin G5
m5	0.4095
clad	0.475
m5

pin G6
m6	0.4095
clad	0.475
m6

pin G7
m7	0.4095
clad	0.475
m7

pin G8
m8	0.4095
clad	0.475
m8

pin G9
m9	0.4095
clad	0.475
m9

pin G10
m10	0.4095
clad	0.475
m10

pin 

## Generating Channels
1. To model an individual channel we will be using the serpent pinStack lattice
2. Each indivial channel will consist of indivial pins (axial layers)
3. There will be 3 types of channels fuel channels, guide tubes, and water gaps

In [5]:
axialheights = np.array([
    -20.000
    ,0.000000
    ,18.28800
    ,36.57600
    ,54.86400
    ,73.15200
    ,91.44000
    ,109.7280
    ,128.0160
    ,146.3040
    ,164.5920
    ,182.8800
    ,201.1680
    ,219.4560
    ,237.7440
    ,256.0320
    ,274.3200
    ,292.6080
    ,310.8960
    ,329.1840
    ,347.4720
    ,365.7600
])

FCPins = fuelPins.copy()
FCPins.append(RB)
FCPins.insert(0, RT)
FCPins = np.array(FCPins)

GTPins = guidePins.copy()
GTPins.append(RB)
GTPins.insert(0, RT)
GTPins = np.array(GTPins)

WGPins = gapPins.copy()
WGPins.append(RB)
WGPins.insert(0, RT)
WGPins = np.array(WGPins)

FC = pinStack("FC", 0, 0, 22)
FC.setStack(FCPins, axialheights)

GT = pinStack("GT", 0, 0, 22)
GT.setStack(GTPins, axialheights)

WG = pinStack("WG", 0, 0, 22)
WG.setStack(WGPins, axialheights)

latList = lats()
latList.addLats([FC, GT, WG])

print(latList.toString())

FC 9 0 0 22
-20.0	RT
0.0	F1
18.288	F2
36.576	F3
54.864	F4
73.152	F5
91.44	F6
109.728	F7
128.016	F8
146.304	F9
164.592	F10
182.88	F11
201.168	F12
219.456	F13
237.744	F14
256.032	F15
274.32	F16
292.608	F17
310.896	F18
329.184	F19
347.472	F20
365.76	RB

GT 9 0 0 22
-20.0	RT
0.0	G1
18.288	G2
36.576	G3
54.864	G4
73.152	G5
91.44	G6
109.728	G7
128.016	G8
146.304	G9
164.592	G10
182.88	G11
201.168	G12
219.456	G13
237.744	G14
256.032	G15
274.32	G16
292.608	G17
310.896	G18
329.184	G19
347.472	G20
365.76	RB

WG 9 0 0 22
-20.0	RT
0.0	GAP1
18.288	GAP2
36.576	GAP3
54.864	GAP4
73.152	GAP5
91.44	GAP6
109.728	GAP7
128.016	GAP8
146.304	GAP9
164.592	GAP10
182.88	GAP11
201.168	GAP12
219.456	GAP13
237.744	GAP14
256.032	GAP15
274.32	GAP16
292.608	GAP17
310.896	GAP18
329.184	GAP19
347.472	GAP20
365.76	RB




## Generating Assembly Layout
1. To model the fuel assembly we will be using a sqaure lattice (19 x 19).
2. The assembly will consist of our generated fuel, guide tube, and water gap channels.

In [6]:
FA = sqLat("FA", 0, 0, 19, 1.260)

FAMap = np.array([
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, GT, FC, FC, FC, FC, FC, FC, FC, FC, FC, GT, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, GT, FC, FC, GT, FC, FC, GT, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, FC, WG],
    [WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG, WG],
])

FA.setMap(FAMap)
print(FA.toString())

FA 0 0  19 1.26
WG WG WG WG WG WG WG WG WG WG WG WG WG WG WG WG WG WG WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC FC FC FC GT FC FC GT FC FC GT FC FC FC FC FC WG 
WG FC FC FC GT FC FC FC FC FC FC FC FC FC GT FC FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC GT FC FC GT FC FC GT FC FC GT FC FC GT FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC GT FC FC GT FC FC GT FC FC GT FC FC GT FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC GT FC FC GT FC FC GT FC FC GT FC FC GT FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG 
WG FC FC FC GT FC FC FC FC FC FC FC FC FC GT FC FC FC WG 
WG FC FC FC FC FC GT FC FC GT FC FC GT FC FC FC FC FC WG 
WG FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC FC WG

## Generating Assembly Border
1. To surrounding surface which encloses the assembly will be modeled using the serpent surface definition
2. Our surrounding surface will be a cuboid enclosing the fuel assembly + surrounding water

In [7]:
width = 10.75 #assembly width
bHeight = -20 #bottom of assembly
tHeight = 385.76 #top of assembly
border = surf("border", "cuboid", np.array([-width, width, -width, width, bHeight, tHeight]))
print(border.toString())

surf border cuboid -10.75 10.75 -10.75 10.75 -20.0 385.76 



## Putting it all together
1. The core/assembly layout (main lattice) must be given.
2. The channels must be given (pinStack objs)
3. The channels axial layers must be given. (pin objs)
4. The materials must be given.
5. The border surface must be given.
6. The optional flags parameter can be set for additional settings

In [8]:
input = inputCollector(FA, latList, pinList, matList, border)
print(input.toString())

AttributeError: 'list' object has no attribute 'toString'